In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import random


















# Load FordA dataset
def load_forda_data():
    train = np.loadtxt("FordA_TRAIN.txt")
    test = np.loadtxt("FordA_TEST.txt")
    
    y_train, X_train = train[:, 0], train[:, 1:]
    y_test, X_test = test[:, 0], test[:, 1:]
    
    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_forda_data()














# Convert data to PyTorch tensors and create DataLoaders
def create_dataloader(X, y, batch_size=32):
    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # add channel dim
    y_tensor = torch.tensor((y > 0).astype(np.int64))  # convert -1/1 to 0/1
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)












# Simple CNN with unique ReLU modules
class SimpleCNN(nn.Module):
    def __init__(self, num_channels=1, num_classes=2, filters1=16, filters2=32, dropout_rate=0.0):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(num_channels, filters1, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(filters1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.conv2 = nn.Conv1d(filters1, filters2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(filters2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(filters2, num_classes)

    def forward(self, x):
        x = self.dropout1(self.relu1(self.bn1(self.conv1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool(x).squeeze(-1)
        return self.fc(x)
    
    
    
    
    
    
    
    
    

# ResNet Block with separate ReLUs
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dropout_rate=0.0):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.shortcut = (
            nn.Conv1d(in_channels, out_channels, kernel_size=1)
            if in_channels != out_channels else None
        )

    def forward(self, x):
        residual = x if self.shortcut is None else self.shortcut(x)
        out = self.dropout1(self.relu1(self.bn1(self.conv1(x))))
        out = self.bn2(self.conv2(out))
        out += residual
        return self.dropout2(self.relu2(out))

class ResNet1D(nn.Module):
    def __init__(self, num_channels=1, num_classes=2, filters1=16, filters2=32, dropout_rate=0.0):
        super(ResNet1D, self).__init__()
        self.layer1 = ResBlock(num_channels, filters1, dropout_rate=dropout_rate)
        self.layer2 = ResBlock(filters1, filters2, dropout_rate=dropout_rate)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(filters2, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.pool(x).squeeze(-1)
        return self.fc(x)
    
    
    
    
    
    
    
    
    
    

# Cross-validation training function
def train_with_cv(model_class, X, y, hyperparams, n_splits=3, device="cpu"):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    cv_scores = []
    
    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        train_loader = create_dataloader(X_train_fold, y_train_fold, batch_size=hyperparams['batch_size'])
        val_loader = create_dataloader(X_val_fold, y_val_fold, batch_size=hyperparams['batch_size'])
        
        # Initialize model with proper hyperparameters
        if model_class.__name__ == 'SimpleCNN' or model_class.__name__ == 'ResNet1D':
            model = model_class(
                filters1=hyperparams['filters1'], 
                filters2=hyperparams['filters2'],
                dropout_rate=hyperparams['dropout_rate']
            )
        else:
            model = model_class()
        
        model.to(device)
        
        # Initialize optimizer based on hyperparams
        if hyperparams['optimizer'] == 'adam':
            optimizer = optim.Adam(
                model.parameters(), 
                lr=hyperparams['lr'], 
                weight_decay=hyperparams['weight_decay'],
                betas=(hyperparams['beta1'], 0.999)
            )
        elif hyperparams['optimizer'] == 'sgd':
            optimizer = optim.SGD(
                model.parameters(), 
                lr=hyperparams['lr'], 
                momentum=hyperparams['momentum'], 
                weight_decay=hyperparams['weight_decay'],
                nesterov=hyperparams['nesterov']
            )
        
        # Scheduler
        if hyperparams['use_scheduler']:
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, 
                mode='max',
                factor=0.5, 
                patience=2
            )
        
        criterion = nn.CrossEntropyLoss()
        
        best_val_acc = 0
        patience_counter = 0
        
        
        
        
        
        
        
        
        
        
        # Training loop
        for epoch in range(hyperparams['epochs']):
            model.train()
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                output = model(X_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                
                # Apply gradient clipping if specified
                if hyperparams['clip_grad_norm'] > 0:
                    nn.utils.clip_grad_norm_(model.parameters(), hyperparams['clip_grad_norm'])
                    
                optimizer.step()
            
            
            
            
            
            
            
            
            # Validation
            model.eval()
            all_preds, all_labels = [], []
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch = X_batch.to(device)
                    preds = model(X_batch).argmax(dim=1).cpu().numpy()
                    all_preds.extend(preds)
                    all_labels.extend(y_batch.numpy())
            
            val_acc = accuracy_score(all_labels, all_preds)
            
            # Update scheduler if used
            if hyperparams['use_scheduler']:
                scheduler.step(val_acc)
            
            # Early stopping
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0
            else:
                patience_counter += 1
                
            if hyperparams['early_stopping'] and patience_counter >= hyperparams['patience']:
                break
        
        cv_scores.append(best_val_acc)
    
    return np.mean(cv_scores)





# Random search for hyperparameter tuning
def random_search(model_class, X, y, param_distributions, n_iter=20, device="cpu"):
    results = []
    
    for i in tqdm(range(n_iter), desc=f"Random Search for {model_class.__name__}"):
        # Randomly sample hyperparameters
        hyperparams = {}
        for param_name, param_dist in param_distributions.items():
            if isinstance(param_dist, list):
                hyperparams[param_name] = random.choice(param_dist)
            elif isinstance(param_dist, tuple) and len(param_dist) == 3:
                # For continuous distributions, we have (min, max, log_scale)
                min_val, max_val, log_scale = param_dist
                if log_scale:
                    hyperparams[param_name] = 10 ** (random.uniform(np.log10(min_val), np.log10(max_val)))
                else:
                    hyperparams[param_name] = random.uniform(min_val, max_val)
        
        # Perform cross-validation
        cv_score = train_with_cv(model_class, X, y, hyperparams, device=device)
        
        # Store results
        hyperparams['cv_score'] = cv_score
        results.append(hyperparams)
        
        print(f"Trial {i+1}/{n_iter}, Score: {cv_score:.4f}, Params: {hyperparams}")
    
    # Convert to DataFrame for easier analysis
    results_df = pd.DataFrame(results)
    
    # Find best hyperparameters
    best_idx = results_df['cv_score'].idxmax()
    best_params = results_df.iloc[best_idx].to_dict()
    
    return results_df, best_params










# Final evaluation on test set
def evaluate_model(model_class, best_params, X_train, y_train, X_test, y_test, device="cpu"):
    # Create data loaders
    train_loader = create_dataloader(X_train, y_train, batch_size=int(best_params['batch_size']))
    test_loader = create_dataloader(X_test, y_test, batch_size=int(best_params['batch_size']))
    
    # Initialize model with best parameters
    if model_class.__name__ == 'SimpleCNN' or model_class.__name__ == 'ResNet1D':
        model = model_class(
            filters1=int(best_params['filters1']), 
            filters2=int(best_params['filters2']),
            dropout_rate=best_params['dropout_rate']
        )
    else:
        model = model_class()
    
    model.to(device)
    
    # Initialize optimizer
    if best_params['optimizer'] == 'adam':
        optimizer = optim.Adam(
            model.parameters(), 
            lr=best_params['lr'], 
            weight_decay=best_params['weight_decay'],
            betas=(best_params['beta1'], 0.999)
        )
    elif best_params['optimizer'] == 'sgd':
        optimizer = optim.SGD(
            model.parameters(), 
            lr=best_params['lr'], 
            momentum=best_params['momentum'], 
            weight_decay=best_params['weight_decay'],
            nesterov=best_params['nesterov']
        )
    
    # Learning rate scheduler
    if best_params['use_scheduler']:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=2
        )
    
    criterion = nn.CrossEntropyLoss()
   





    # Train the model with best parameters
    best_test_acc = 0
    for epoch in range(int(best_params['epochs'])):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            
            # Apply gradient clipping if specified
            if best_params['clip_grad_norm'] > 0:
                nn.utils.clip_grad_norm_(model.parameters(), best_params['clip_grad_norm'])
                
            optimizer.step()
        
        # Eval on test set
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch = X_batch.to(device)
                preds = model(X_batch).argmax(dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(y_batch.numpy())
        acc = accuracy_score(all_labels, all_preds)
        
        if best_params['use_scheduler']:
            scheduler.step(acc)
            
        if acc > best_test_acc:
            best_test_acc = acc
            # Save the best model
            torch.save(model.state_dict(), f"best_{model_class.__name__}_model.pt")
            
        print(f"Epoch {epoch+1}/{int(best_params['epochs'])}, Test Accuracy: {acc:.4f}")
    
    # Load best model for final evaluation
    model.load_state_dict(torch.load(f"best_{model_class.__name__}_model.pt"))
    
    # Final evaluation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch).argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y_batch.numpy())
    
    final_acc = accuracy_score(all_labels, all_preds)
    print(f"\nFinal Test Accuracy with best parameters: {final_acc:.4f}")
    return model, final_acc

# Visualize results
def plot_hyperparameter_results(results_df, model_name):
    # Select the top parameters for visualization
    top_params = ['lr', 'batch_size', 'filters1', 'filters2', 'dropout_rate', 'weight_decay']
    
    plt.figure(figsize=(15, 10))
    
    for i, param in enumerate(top_params):
        if param in results_df.columns:
            plt.subplot(2, 3, i+1)
            plt.scatter(results_df[param], results_df['cv_score'])
            
            # Add trendline for continuous parameters
            if len(results_df[param].unique()) > 10:
                try:
                    z = np.polyfit(results_df[param], results_df['cv_score'], 1)
                    p = np.poly1d(z)
                    plt.plot(sorted(results_df[param]), p(sorted(results_df[param])), "r--")
                except:
                    pass
                
                # For log-scale parameters
                if param in ['lr', 'weight_decay'] and results_df[param].min() > 0:
                    plt.xscale('log')
            
            plt.xlabel(param)
            plt.ylabel('CV Accuracy')
            plt.title(f'{param} vs CV Accuracy')
    
    plt.tight_layout()
    plt.savefig(f'{model_name}_hyperparameter_results.png')
    plt.show()
    
    # Plot performance distribution
    plt.figure(figsize=(10, 6))
    plt.hist(results_df['cv_score'], bins=20)
    plt.xlabel('CV Accuracy')
    plt.ylabel('Frequency')
    plt.title(f'{model_name} Performance Distribution')
    plt.axvline(results_df['cv_score'].max(), color='r', linestyle='--', 
                label=f'Best Score: {results_df["cv_score"].max():.4f}')
    plt.legend()
    plt.savefig(f'{model_name}_performance_distribution.png')
    plt.show()

# Main execution
if __name__ == "__main__":
    # Set random seed for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    
    # Device setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Define parameter distributions for random search
    param_distributions = {
        # Discrete parameters
        'batch_size': [8, 16, 32, 64, 128],
        'epochs': [10, 15, 20, 30],
        'filters1': [16, 32, 64, 128],
        'filters2': [32, 64, 128, 256],
        'optimizer': ['adam', 'sgd'],
        'use_scheduler': [True, False],
        'early_stopping': [True, False],
        'patience': [3, 5, 7],
        'nesterov': [True, False],
        
        # Continuous parameters (min, max, log_scale)
        'lr': (1e-5, 1e-1, True),
        'weight_decay': (1e-6, 1e-3, True),
        'dropout_rate': (0.0, 0.5, False),
        'momentum': (0.8, 0.99, False),
        'beta1': (0.8, 0.99, False),
        'clip_grad_norm': (0.0, 5.0, False),
    }
    
    # Number of random trials
    n_trials = 5  # Adjust based on available computational resources
    
    # Perform random search for SimpleCNN
    print("\n=== Random Search for SimpleCNN ===")
    cnn_results, cnn_best_params = random_search(SimpleCNN, X_train, y_train, param_distributions, n_iter=n_trials, device=device)
    print(f"\nBest parameters for SimpleCNN: {cnn_best_params}")
    plot_hyperparameter_results(cnn_results, "SimpleCNN")
    
    # Perform random search for ResNet1D
    print("\n=== Random Search for ResNet1D ===")
    resnet_results, resnet_best_params = random_search(ResNet1D, X_train, y_train, param_distributions, n_iter=n_trials, device=device)
    print(f"\nBest parameters for ResNet1D: {resnet_best_params}")
    plot_hyperparameter_results(resnet_results, "ResNet1D")
    
    # Evaluate models with best parameters
    print("\n=== Evaluating SimpleCNN with best parameters ===")
    best_cnn_model, cnn_acc = evaluate_model(SimpleCNN, cnn_best_params, X_train, y_train, X_test, y_test, device=device)
    
    print("\n=== Evaluating ResNet1D with best parameters ===")
    best_resnet_model, resnet_acc = evaluate_model(ResNet1D, resnet_best_params, X_train, y_train, X_test, y_test, device=device)
    
    # Compare models
    print("\n=== Model Comparison ===")
    print(f"SimpleCNN Test Accuracy: {cnn_acc:.4f}")
    print(f"ResNet1D Test Accuracy: {resnet_acc:.4f}")
    
    # Save best hyperparameters
    cnn_results.to_csv('simplecnn_random_search_results.csv', index=False)
    resnet_results.to_csv('resnet1d_random_search_results.csv', index=False)
    
    # Print summary of best configurations
    print("\n=== Best Configurations ===")
    print("SimpleCNN:")
    for param, value in cnn_best_params.items():
        print(f"  {param}: {value}")
    
    print("\nResNet1D:")
    for param, value in resnet_best_params.items():
        print(f"  {param}: {value}")

Using device: cpu

=== Random Search for SimpleCNN ===


Random Search for SimpleCNN:   0%|          | 0/5 [00:29<?, ?it/s]


KeyboardInterrupt: 